In [1]:
from pyspark import SparkContext

In [2]:
sc = SparkContext(master="local", appName="transformacionesyAcciones")

22/03/27 23:05:23 WARN Utils: Your hostname, luishm resolves to a loopback address: 127.0.1.1; using 192.168.0.117 instead (on interface wlp7s0)
22/03/27 23:05:23 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/03/27 23:05:24 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/03/27 23:05:25 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [7]:
rdd1 = sc.parallelize([1,2,3])
type(rdd1)

pyspark.rdd.RDD

In [5]:
rdd1.collect()

[1, 2, 3]

In [7]:
!ls ../../curso-apache-spark-platzi/files/

deporte.csv	 deportistaError.csv  modelo_relacional.jpg
deportista2.csv  evento.csv	      paises.csv
deportista.csv	 juegos.csv	      resultados.csv


In [8]:
path = "../../curso-apache-spark-platzi/files/"

In [9]:
equiposOlimpicosrdd = sc.textFile(path+"paises.csv") \
                      .map(lambda line : line.split(","))

In [10]:
equiposOlimpicosrdd.take(15)

[['id', 'equipo', 'sigla'],
 ['1', '30. Februar', 'AUT'],
 ['2', 'A North American Team', 'MEX'],
 ['3', 'Acipactli', 'MEX'],
 ['4', 'Acturus', 'ARG'],
 ['5', 'Afghanistan', 'AFG'],
 ['6', 'Akatonbo', 'IRL'],
 ['7', 'Alain IV', 'SUI'],
 ['8', 'Albania', 'ALB'],
 ['9', 'Alcaid', 'POR'],
 ['10', 'Alcyon-6', 'FRA'],
 ['11', 'Alcyon-7', 'FRA'],
 ['12', 'Aldebaran', 'ITA'],
 ['13', 'Aldebaran II', 'ITA'],
 ['14', 'Aletta', 'IRL']]

In [11]:
equiposOlimpicosrdd.map(lambda x: (x[2])).distinct().count()

231

In [12]:
equiposOlimpicosrdd.map(lambda x: (x[2], x[1])).groupByKey().mapValues(len).take(5)

[('sigla', 1), ('AUT', 11), ('MEX', 9), ('ARG', 18), ('AFG', 1)]

In [13]:
equipos_argentinos = equiposOlimpicosrdd.filter(lambda l: "MEX" in l)
equipos_argentinos.collect()

[['2', 'A North American Team', 'MEX'],
 ['3', 'Acipactli', 'MEX'],
 ['193', 'Chamukina', 'MEX'],
 ['656', 'Mexico', 'MEX'],
 ['657', 'Mexico-1', 'MEX'],
 ['658', 'Mexico-2', 'MEX'],
 ['700', 'Nausikaa 4', 'MEX'],
 ['1053', 'Tlaloc', 'MEX'],
 ['1168', 'Xolotl', 'MEX']]

In [14]:
equiposOlimpicosrdd.count()

1185

In [15]:
equiposOlimpicosrdd.countApprox(20)

1185

In [3]:
! ls ../files

deporte.csv	 deportistaError.csv  modelo_relacional.jpg
deportista2.csv  evento.csv	      paises.csv
deportista.csv	 juegos.csv	      resultados.csv


In [16]:
deportistaOlimpicoRDD = sc.textFile(path+"deportista.csv") \
                        .map(lambda l: l.split(","))
deportistaOlimpicoRDD2 = sc.textFile(path+"deportista2.csv") \
                        .map(lambda l: l.split(","))

In [17]:
deportistaOlimpicoRDD=deportistaOlimpicoRDD \
                      .union(deportistaOlimpicoRDD2)

In [18]:
deportistaOlimpicoRDD.count()

135572

In [19]:
equiposOlimpicosrdd.top(2)

[['id', 'equipo', 'sigla'], ['999', 'Stella-2', 'NOR']]

In [20]:
deportistaOlimpicoRDD.top(2)

[['deportista_id', 'nombre', 'genero', 'edad', 'altura', 'peso', 'equipo_id'],
 ['99999', 'Alexander Grant Alick Rennie', '1', '32', '182', '71', '967']]

In [51]:
 deportistaOlimpicoRDD.map(lambda l: [l[-1], l[:-1]]) \
                    .join(equiposOlimpicosrdd.map(lambda x : [x[0], x[2]])) \
                    .take(5000)

[('199', (['1', 'A Dijiang', '1', '24', '180', '80'], 'CHN')),
 ('199', (['2', 'A Lamusi', '1', '23', '170', '60'], 'CHN')),
 ('199', (['602', 'Abudoureheman', '1', '22', '182', '75'], 'CHN')),
 ('199', (['1463', 'Ai Linuer', '1', '25', '160', '62'], 'CHN')),
 ('199', (['1464', 'Ai Yanhan', '2', '14', '168', '54'], 'CHN')),
 ('199', (['3605', 'An Weijiang', '1', '22', '178', '72'], 'CHN')),
 ('199', (['3610', 'An Yulong', '1', '19', '173', '70'], 'CHN')),
 ('199', (['3611', 'An Zhongxin', '2', '23', '170', '65'], 'CHN')),
 ('199', (['4639', 'Ao Changrong', '1', '25', '173', '71'], 'CHN')),
 ('199', (['4641', 'Ao Tegen', '1', '21', '181', '90'], 'CHN')),
 ('199', (['6376', 'Ba Dexin', '1', '23', '185', '80'], 'CHN')),
 ('199', (['6381', 'Ba Yan', '2', '21', '183', '78'], 'CHN')),
 ('199', (['6382', 'Ba Yanchuan', '1', '24', '187', '100'], 'CHN')),
 ('199', (['6383', 'Ba Yongshan', '1', '22', '180', '70'], 'CHN')),
 ('199', (['6847', 'Bai Anqi', '2', '19', '164', '59'], 'CHN')),
 ('199',

In [26]:
resultadosOlimpicosRDD = sc.textFile(path+"resultados.csv").map(lambda l: l.split(","))
resultadosOlimpicosRDD.take(2)

[['resultado_id', 'medalla', 'deportista_id', 'juego_id', 'evento_id'],
 ['1', 'NA', '1', '39', '1']]

In [27]:
resultadosOlimpicosRDD = resultadosOlimpicosRDD.filter(lambda l: 'NA' not in l[1])
resultadosOlimpicosRDD.take(2)

[['resultado_id', 'medalla', 'deportista_id', 'juego_id', 'evento_id'],
 ['4', 'Gold', '4', '2', '4']]

In [55]:
deportistaOlimpicoRDD.map(lambda l: [l[-1], l[:-1]]) \
                    .join(equiposOlimpicosrdd.map(lambda x : [x[0], x[2]])) \
                    .map(lambda y: [y[1][0][0], [y[1][0][1], y[1][1]]]) \
                    .join(resultadosOlimpicosRDD.map(lambda z: [z[2], z[1]])) \
                    .take(500)

[('7597', (['Bao Yingying', 'CHN'], 'Silver')),
 ('17282', (['Cai Huijue', 'CHN'], 'Bronze')),
 ('17996', (['Cao Mianying', 'CHN'], 'Silver')),
 ('19779', (['Chang Si', 'CHN'], 'Silver')),
 ('19791', (['Chang Yongxiang', 'CHN'], 'Silver')),
 ('20215', (['Chen Jin', 'CHN'], 'Bronze')),
 ('20217', (['Chen Jing', 'CHN'], 'Gold')),
 ('20217', (['Chen Jing', 'CHN'], 'Silver')),
 ('20217', (['Chen Jing', 'CHN'], 'Silver')),
 ('20217', (['Chen Jing', 'CHN'], 'Bronze')),
 ('20306', (['Chen Yan', 'CHN'], 'Bronze')),
 ('20315', (['Chen Ying', 'CHN'], 'Gold')),
 ('20315', (['Chen Ying', 'CHN'], 'Silver')),
 ('20324', (['Chen Yueling', 'CHN'], 'Gold')),
 ('20335', (['Chen Zhong', 'CHN'], 'Gold')),
 ('20335', (['Chen Zhong', 'CHN'], 'Gold')),
 ('20337', (['Chen Zihe', 'CHN'], 'Silver')),
 ('20374', (['Cheng Ming', 'CHN'], 'Silver')),
 ('24911', (['Dai Jun', 'CHN'], 'Bronze')),
 ('28663', (['Ding Xia', 'CHN'], 'Gold')),
 ('29377', (['Dong Jiong', 'CHN'], 'Silver')),
 ('29382', (['Dong Zhaozhi', 'CHN

In [41]:
 deportistasJoinResultados = deportistaOlimpicoRDD.map(lambda l: [l[-1], l[:-1]]) \
                    .join(equiposOlimpicosrdd.map(lambda x : [x[0], x[2]])) \
                    .map(lambda y: [y[1][0][0], [y[1][0][1], y[1][1]]]) \
                    .join(resultadosOlimpicosRDD.map(lambda z: [z[2], z[1]]))

In [42]:
valorMedallas = {
'Gold':7,
'Silver': 5,
'Bronze': 4}

In [59]:
datosmedallas = deportistasJoinResultados.map(lambda x: [x[1][0][1], valorMedallas[x[1][1]]])

In [62]:
from operator import add
data_conclusion = datosmedallas.reduceByKey((add)).sortBy(lambda x : x[1], ascending = False)
data_conclusion.take(10)

[('USA', 32137),
 ('URS', 14834),
 ('GBR', 10925),
 ('GER', 10896),
 ('FRA', 9265),
 ('ITA', 8755),
 ('SWE', 8110),
 ('CAN', 7209),
 ('AUS', 6755),
 ('HUN', 6142)]